In [2]:
import pandas as pd
import numpy as np
import requests

## load golang vulneribility database

In [28]:
def load_golang_vul_db():
    vul_db_url = "https://vuln.go.dev/index/modules.json"
    resp = requests.get(vul_db_url)
    vul_dicts = []    
    for m in resp.json():
        for v in m["vulns"]:
            vul_info = _load_vul_info(v["id"])
            vul_dict = {
                "module":  m["path"],
                "vul_id":  v["id"],
                "last_updated":  v["modified"],
                "fix_version":  v.get("fixed", ""),
                "summary":  vul_info["summary"],
                "details":  vul_info["details"],
                "aliases":  vul_info["aliases"],
                "published":  vul_info["published"],
            }
            vul_dicts.append(vul_dict)
    return pd.DataFrame(vul_dicts)    

def _load_vul_info(id):
    request_url = f"https://vuln.go.dev/ID/{id}.json"
    resp = requests.get(request_url)
    dikt = resp.json()
    vul_info = {
       "summary": dikt.get("summary", ""), 
       "details": dikt.get("details", ""), 
       "published": dikt.get("published", ""), 
       "aliases": dikt.get("aliases", []), 
    }
    return vul_info


In [ ]:
%%time
df = load_golang_vul_db()
df

In [34]:
df['last_updated'] = pd.to_datetime(df['last_updated'])
df['published'] = pd.to_datetime(df['published'])

In [51]:
df['alias'] = [x[0] if len(x) >= 1 else '' for x in df["aliases"]]

In [57]:
df = df.sort_values("published", ascending=False)
df.to_csv('govuls.csv', index=False, columns=["module", "vul_id", "alias", "published", "last_updated", "fix_version", "summary"])
